### Optimización de Turbogeneradores de Vapor

Se presenta el análisis y construcción del problema de optimización de *Maximización de Eficiencia de Producción de Energía* en Turbogeneradores de Ingenio Pantaleon. El objetivo es determinar recomendaciones sobre qué Turbogeneradores operar y a qué carga dada una demanda de vapor específica, de forma que:
- Se maximice la cantidad de energía producida dada una demanda de vapor específica. 
- Se opere dentro de los límites máximos y mínimos de carga de los turbogeneradores.

In [1]:
import numpy as np
import pandas as pd
from gurobipy import *

### Selección de Turbogeneradores

Basado en su Potencia Nominal Máxima, se construye un problema de optimización para seleccionar los turbogeneradores más eficiente para una demanda de energía específica.

#### Conjuntos y Notaciones

Se define la siguiente notación:

- $P_i$ la *Potencia* (en kW) generada en el Turbogenerador $i$, $\forall i \in \{1,2,3,4,5,6\}$

- $F_i$ la *Eficiencia* (en klb Vapor/kW) del Turbogenerador $i$, $\forall i \in \{1,2,3,4,5,6\}$

- $C_i$ el *Consumo de Vapor* (en klb Vapor) del Turbogenerador $i$, $\forall i \in \{1,2,3,4,5,6\}$

De forma que:

$$ C_i = P_i * F_i, \forall i \in \{1,2,3,4,5,6\}$$

##### Variables de Decisión
- $x_i$ si el Turbogenerador $i$ debe o no estar en linea, $x_i \in \{0,1\}, \forall i \in \{1,2,3,4,5,6\}$

#### Función Objetivo

El objetivo es minimizar es el *Consumo Total de Vapor* dada una *Potencia Total Demandada*:
$$ \min \sum_{i=1}^{6} C_i = \sum_{i=1}^{6} x_i * P_i * F_i  $$

Sujeta a:

- $\sum_{i=1}^{6} x_i * P_i \geq P_{Demanda}$ (La potencia total generada debe ser mayor o igual que la potencia total requerida).
- $x_i \in \{0,1\}$

In [2]:
DEMANDA = 61230
df = pd.read_csv(r'Data/Datos Turbos.csv')
df

,Turbogenerador,Potencia Maxima (kW),Potencia Minima (kW),Eficiencia (lb vapor / kW)
0,1,22000,4000,10
1,2,21000,5000,12
2,3,15000,3000,10
3,4,8000,4000,11
4,5,10000,5000,10
5,6,20000,3000,12


In [3]:
# turbogeneradores
turbos = np.array(df)

# número de turbogeneradores
n = len(df['Turbogenerador'])

# lista de turbogeneradores
N = {i for i in range(1,n+1)}

# parámetros
Pot_Max = {i:turbos[i-1,1] for i in N}
Pot_Min = {i:turbos[i-1,2] for i in N}
Eff = {i:turbos[i-1,3] for i in N}

In [4]:
# modelo de optimización
model = Model("Seleccion de Turbogeneradores")

# variables de decisión
x = model.addVars(N, vtype = GRB.BINARY)

Set parameter Username
Set parameter LicenseID to value 2647918


In [5]:
# función objetivo
model.setObjective(quicksum(Pot_Max[i]*Eff[i]*x[i] for i in N), GRB.MINIMIZE)

In [6]:
# restricciones
model.addConstr(quicksum(Pot_Max[i]*x[i]for i in N)>=DEMANDA)
model.update()

In [7]:
# optimización
model.optimize()

if model.status != GRB.status.OPTIMAL:
    print("No se encontró una solución factible...")
else:
    print("El valor optimo de la función objetivo es %g"%model.objVal)
    resultado_objetivo = model.objVal/1000
    resultado_seleccion_turbos = [i for i in N if x[i].X >= 1]
    resultado_seleccion_potencia = np.sum([Pot_Max[i] for i in N if x[i].X >= 1])

    df_salida = pd.DataFrame({"consumo de vapor máximo total (klb)": [resultado_objetivo],
                              "turbogeneradores seleccionados": [resultado_seleccion_turbos],
                              "potencia total máxima entregada (kW)": [resultado_seleccion_potencia]})

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.4.0 24E248)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 1 rows, 6 columns and 6 nonzeros
Model fingerprint: 0xc44a7570
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [8e+03, 2e+04]
  Objective range  [9e+04, 3e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e+04, 6e+04]
Found heuristic solution: objective 742000.00000
Presolve removed 1 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 10 available processors)

Solution count 2: 698000 742000 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.980000000000e+05, best bound 6.980000000000e+05, gap 0.0000%
El valor optimo de la función objetivo es 698000


In [8]:
df_salida

,consumo de vapor máximo total (klb),turbogeneradores seleccionados,potencia total máxima entregada (kW)
0,698.0,"[1, 3, 4, 6]",65000


### Carga de Turbogeneradores

Una vez se definen los turbos a operar, se optimiza esa configuración para recomendar los niveles de carga que permitan hacer más eficiente la generación.

#### Conjuntos y Notaciones

Se define la siguiente notación:

- Sea $turbos-seleccionados$ el conjunto de turbogeneradores que deben operarse para satisfacer la demanda de energía.

- Sea $x_i$ la *Potencia* (en kW) generada en el Turbogenerador $i$, $\forall i \in \{turbos-seleccionados\}$

- Sea $F_i$ la *Eficiencia* (en klb Vapor/kW) del Turbogenerador $i$, $\forall i \in \{turbos-seleccionados\}$

- Sea $C_i$ el *Consumo de Vapor* (en klb Vapor) del Turbogenerador $i$, $\forall i \in \{turbos-seleccionados\}$

De forma que:

$$ C_i = x_i * F_i, \forall i \in \{turbos-seleccionados\}$$

##### Variables de Decisión
- $x_i$ la Potencia Requerida al Turbogenerador $i$, $x_i \in \mathbb{R}_{\geq 0}, \forall i \in \{turbos-seleccionados\}$

#### Función Objetivo

El objetivo es minimizar es el *Consumo Total de Vapor* dada una *Potencia Total Demandada*:
$$ \min \sum_{i=1}^{6} C_i = \sum_{i=1}^{6} x_i * F_i  $$

Sujeta a:

- $\sum_{i=1}^{6} x_i \geq P_{Demanda}$ (La potencia total generada debe ser mayor o igual que la potencia total requerida).
- $x_i \geq P_{i-min}, \forall i \in \{turbos-seleccionados\}$ (La potencia generada por el turbogenerador $i$ no sea inferior a su potencia mínima nominal)
- $x_i \leq P_{i-max}, \forall i \in \{turbos-seleccionados\}$ (La potencia generada por el turbogenerador $i$ no supere su potencia máxima nominal)
- $x_i \in \mathbb{R}_{\geq 0}, \forall i \in \{turbos-seleccionados\}$

In [9]:
resultado_seleccion_turbos

[1, 3, 4, 6]

In [10]:
df = pd.read_csv(r'Data/Datos Turbos.csv')

# Se filtran los turbogeneradores seleccionados
mask = df['Turbogenerador'].isin(resultado_seleccion_turbos)
df_turbos = df[mask]
df_turbos

,Turbogenerador,Potencia Maxima (kW),Potencia Minima (kW),Eficiencia (lb vapor / kW)
0,1,22000,4000,10
2,3,15000,3000,10
3,4,8000,4000,11
5,6,20000,3000,12


In [11]:
# turbogeneradores
turbos = np.array(df)

# número de turbogeneradores
n = len(df_turbos['Turbogenerador'])

# lista de turbogeneradores
N = {i for i in range(1,n+1)}

# parámetros
Pot_Max = {i:turbos[i-1,1] for i in N}
Pot_Min = {i:turbos[i-1,2] for i in N}
Eff = {i:turbos[i-1,3] for i in N}

In [12]:
# modelo de optimización
model = Model("Seleccion de Turbogeneradores")

# variables de decisión
x = model.addVars(N, vtype = GRB.CONTINUOUS)

In [13]:
# función objetivo
model.setObjective(quicksum(x[i]*Eff[i] for i in N), GRB.MINIMIZE)

In [14]:
# restricciones
model.addConstr(quicksum(x[i]for i in N)>=DEMANDA)
model.addConstrs(x[i] <= Pot_Max[i] for i in N)
model.addConstrs(x[i] >= Pot_Min[i] for i in N)
model.update()

In [15]:
# optimización
model.optimize()

if model.status != GRB.status.OPTIMAL:
    print("No se encontró una solución factible...")
else:
    print("El valor optimo de la función objetivo es %g"%model.objVal)
    resultado_objetivo = model.objVal/1000
    resultado_carga_turbos = [x[i].X for i in N]
    resultado_seleccion_potencia = np.sum([x[i].X for i in N])

    df_salida = pd.DataFrame({"consumo de vapor proyectado (klb)": [resultado_objetivo],
                               "turbogeneradores seleccionados": [resultado_seleccion_turbos],
                              "carga de turbogeneradores seleccionados (kW)": [resultado_carga_turbos],
                              "potencia total proyectada (kW)": [resultado_seleccion_potencia]})

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.4.0 24E248)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 9 rows, 4 columns and 12 nonzeros
Model fingerprint: 0x413e78cc
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+03, 6e+04]
Presolve removed 9 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.5276000e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  6.527600000e+05
El valor optimo de la función objetivo es 652760


In [16]:
df_salida

,consumo de vapor proyectado (klb),turbogeneradores seleccionados,carga de turbogeneradores seleccionados (kW),potencia total proyectada (kW)
0,652.76,"[1, 3, 4, 6]","[22000.0, 16230.0, 15000.0, 8000.0]",61230.0
